In [1]:
import pandas as pd
pd.options.display.max_rows=1000
import numpy as np

# Load and format data

In [2]:
df = pd.read_excel('/Users/nathansuberi/Downloads/retailreport2.xls')
df = df.reset_index()
df = df.dropna(axis=0, how='all')
df = df.drop([1,6,8,10])
df.columns = df.loc[12]
df = df.drop([12,15])
df = df.dropna(axis=1, how='all')

In [8]:
df.columns = ['DCS Code',
 'Vendor\nCode',
 'Description1',
 None,
 'Attribute',
 None,
 'Size',
 'UPC',
 'QTY',
 None,
 'Ext Price',
 'Ext Cost',
 'Ext Margin Amount',
 'Margin %']

df.drop(None, axis=1)

In [ ]:
trimmed_df = df.drop([12192, 12194, 12197])

In [ ]:
sandal_flag = [a or b for a,b in zip(trimmed_df['DCS Code'].str.contains('W  S'), trimmed_df['DCS Code'].str.contains('M  S'))]
sandals = df.loc[sandal_flag]

In [ ]:
sandals

In [ ]:
grouped = df.groupby(['Vendor\nCode','Description1', 'Attribute'])
sums = grouped.sum()
vendors = list(sums.index.levels[0])

# For output to HTML

In [ ]:
with open('/Users/nathansuberi/Desktop/enchanted_reports/summer_sale_2018/summer_sale_reference.html', 'w') as f:
    for vendor in vendors:
        vendor_data = sums.loc[vendor, :, :].reset_index()
        sale_flag = vendor_data['QTY'] < -5
        on_sale = vendor_data.loc[sale_flag, ['Description1', 'Attribute', 'QTY']].reset_index().drop('index', axis=1)
        not_on_sale = vendor_data.loc[~sale_flag, ['Description1', 'Attribute', 'QTY']].reset_index().drop('index', axis=1)
        
        if not_on_sale.shape[0]:
            f.write('<h1>Vendor: ' + vendor + '</h1>\n')
            last_style = ''
            for line in not_on_sale.values:
                style = line[0]
                color = line[1]
                qty = int(line[2])
                if last_style == style:
                    f.write('<i>{}, {}, sold: <font size=4>{}</font>'.format(style, color, qty) + '</i> <br/>')
                else:
                    f.write('<br/><b>{}, {}, sold: <font size=4>{}</font>'.format(style, color, qty) + '</b> <br/>')
                last_style = style

# For output to csvs

In [ ]:
for vendor in vendors:
    vendor_data = sums.loc[vendor, :, :].reset_index()
    sale_flag = vendor_data['QTY'] < 4
    on_sale = vendor_data.loc[sale_flag, ['Description1', 'Attribute', 'QTY']].reset_index().drop('index', axis=1)
    not_on_sale = vendor_data.loc[~sale_flag, ['Description1', 'Attribute', 'QTY']].reset_index().drop('index', axis=1)

    if on_sale.shape[0]:
        on_sale.to_csv('/Users/nathansuberi/Desktop/enchanted_reports/summer_sale_2018/onsale_{}.csv'.format(vendor), header=None)
    if not_on_sale.shape[0]:
        not_on_sale.to_csv('/Users/nathansuberi/Desktop/enchanted_reports/summer_sale_2018/notonsale_{}.csv'.format(vendor), header=None)